# pyUmbral Python API

## Generate Umbral Keys for Alice
First, Let's generate two asymmetric key pairs for Alice:
A *delegating* key pair and a *signing* key pair.


In [2]:
from umbral import SecretKey, Signer


# Alice's Keys
alices_private_key = SecretKey.random()
alices_public_key = alices_private_key.public_key()

alices_signing_key = SecretKey.random()
alices_verifying_key = alices_signing_key.public_key()
alices_signer = Signer(alices_signing_key)

## Encrypt some data for Alice
Now let's encrypt data with Alice's public key. Invocation of `pre.encrypt` returns both the `ciphertext`,
and a `capsule`. Anyone with Alice's public key can perform this operation.

In [3]:
from umbral import encrypt


plaintext = b'Proxy Re-encryption is cool!'
capsule, ciphertext = encrypt(alices_public_key, plaintext)
print(ciphertext)

b'\xfb\xc3T\xb2\x89=\x08X\xb1<\xd0G/\xab\x8c\xac\x7f\xd4)\xcbB\xcb^\x99;P\x9c\xbf\xaaf\x03\xdd\n\x1f$\x1b\xfb\x88\xfa\xcd\xe2\x11\x8d\xcf\xe5\x88\xaf\x00\xfe\xcb\x9d\xf83\x17\x9b\xdd\xba\xab\x8b\x08\xbe\xb1M\x80\xf1<S#'


## Alice decrypts data for self
Since data was encrypted with Alice's public key, Alice can open the capsule and decrypt the ciphertext with her private key.

In [4]:
from umbral import decrypt_original


cleartext = decrypt_original(delegating_sk=alices_private_key,
                             capsule=capsule,
                             ciphertext=ciphertext)
print(cleartext)

b'Proxy Re-encryption is cool!'


## Enter Bob
Apart from generating his keypair, we will also assume that Bob receives a capsule through a side channel (s3, ipfs, Google Cloud, etc). 

In [5]:
bobs_private_key = SecretKey.random()
bobs_public_key = bobs_private_key.public_key()

bob_capsule = capsule

## Attempt Bob's decryption (fail)

In [6]:
try:
    fail_decrypted_data = decrypt_original(delegating_sk=bobs_private_key,
                                           capsule=capsule,
                                           ciphertext=ciphertext)
except ValueError:
    print("Decryption failed! Bob doesn't has access granted yet.")


Decryption failed! Bob doesn't has access granted yet.


# Proxy Re-encryption

<img src="https://cdn-images-1.medium.com/max/1200/0*yTKUeeuKPu-aIZdw." alt="Proxy Re-Encryption" width="500"/>

## Alice grants access to Bob by generating KFrags 
When Alice wants to grant Bob access to open her encrypted messages, she creates *re-encryption key fragments*, or "kfrags", which are next sent to N proxies or *Ursulas*. She uses her private key, and Bob's public key, and she sets a minimum threshold of 10, for 20 total shares


In [7]:
from umbral import generate_kfrags


M, N = 10, 20 # the threshold and the total number of fragments
kfrags = generate_kfrags(delegating_sk=alices_private_key,
                         receiving_pk=bobs_public_key,
                         signer=alices_signer,
                         threshold=M,
                         num_kfrags=N)


## Ursulas Re-encrypt; Bob attaches fragments to `capsule`
Bob asks several Ursulas to re-encrypt the capsule so he can open it. Each Ursula performs re-encryption on the capsule using the `kfrag` provided by Alice, obtaining this way a "capsule fragment", or `cfrag`. Let's mock a network or transport layer by sampling `M` random `kfrags`, one for each required Ursula. Bob collects the resulting `cfrags` from several Ursulas. He must gather at least `M` `cfrags` in order to activate the capsule.


In [8]:
import random
kfrags = random.sample(kfrags,  # All kfrags from above
                       M)       # Threshold


from umbral import reencrypt


cfrags = list()                 # Bob's cfrag collection
for kfrag in kfrags:
    cfrag = reencrypt(capsule=capsule, kfrag=kfrag)
    cfrags.append(cfrag)        # Bob collects a cfrag

## Bob checks the capsule fragments
If Bob received the capsule fragments in serialized form, he can verify that they are valid and really originate from Alice, using Alice's public keys.

In [10]:
from umbral import CapsuleFrag

suspicious_cfrags = [CapsuleFrag.from_bytes(bytes(cfrag)) for cfrag in cfrags]

cfrags = [cfrag.verify(capsule,
                       verifying_pk=alices_verifying_key,
                       delegating_pk=alices_public_key,
                       receiving_pk=bobs_public_key,
                       )
          for cfrag in suspicious_cfrags]

## Bob opens the capsule; Decrypts data from Alice.
Finally, Bob decrypts the re-encrypted ciphertext using his secret key.

In [12]:
from umbral import decrypt_reencrypted

bob_cleartext = decrypt_reencrypted(receiving_sk=bobs_private_key,
                                    delegating_pk=alices_public_key,
                                    capsule=capsule,
                                    verified_cfrags=cfrags,
                                    ciphertext=ciphertext)

print(bob_cleartext)
assert bob_cleartext == plaintext

b'Proxy Re-encryption is cool!'
